In [4]:
import os
import requests
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama # type: ignore
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


load_dotenv("../.env")

True

In [5]:
openai = OpenAI()

In [6]:

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

### 1-) Organize Prompts

In [8]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [9]:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
print(user_prompt_for(ed))

In [11]:

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [12]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, posi

### 2-) Use OpenAI for website content summarization

In [15]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [16]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

The website belongs to Ed Donner, who is a programmer, AI enthusiast, and the co-founder and CTO of **Nebula.io**. His work focuses on leveraging AI to help individuals discover their potential and improves talent acquisition through proprietary language model technology. 

## Key Features:
- **Outsmart**: An interactive arena where large language models (LLMs) compete in strategy and negotiation.
- **Personal Interests**: Ed shares his interests in coding, DJing, electronic music production, and staying updated with technology discussions.

## News and Announcements:
- **November 13, 2024**: Released resources for "Mastering AI and LLM Engineering."
- **October 16, 2024**: Provided resources for transitioning from Software Engineer to AI Data Scientist.
- **August 6, 2024**: Announced the "Outsmart LLM Arena."
- **June 26, 2024**: Shared a toolkit and resources for choosing the right LLM.

Ed encourages readers to connect with him for collaboration and further discussion on these topics.